In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

option = webdriver.ChromeOptions()
driver = webdriver.Chrome(options = option)

driver.get('https://www.google.com/')

In [3]:
def make_request(driver, keyword):
    URL = "https://www.altodescuentos.com/business/search?q=" + keyword.strip().replace(' ', '+')
    try:
        driver.get(URL)
    except Exception:
        return None
    html = driver.page_source
    soup = BeautifulSoup(html)
    return soup

In [4]:
def parse_page(soup, keyword):
    cols=['parse_dt', 'partner_id', 'advertiser_name', 'advertiser_pos', 'title', 'content', 'keyword', 'depth']
    df = pd.DataFrame(columns=cols)
    # part 1
    sect = soup.find_all('section')
    for i in range(len(sect)):
        try:
            art = sect[i].find_all("article", {'class':'yh-list__item'})
            if len(art) == 0:
                continue
            partner_id = int(sect[i]['data-item-feed-partner-id'])

            for j in range(len(art)):
                pos = art[j]['data-feed-item-position']
                
                title_block = art[j].find_all('h3', class_='yh-list__title')
                for t in title_block:
                    title = re.sub("<b>|</b>|\'|\,|\[|\]", '',str(t.find('a').contents))
                    
                content_block = art[j].find_all('div', class_='yh-list__descr')
                for c in content_block:
                    content = re.sub("<b>|</b>|\'|\,|\[|\]", '',str(c.find('a').contents))

                urls = art[j].find_all('div', class_='yh-list__url')
                for u in urls:
                    url = 'https://'+ u.find('a').contents[0]
            try:
                row = [datetime.now().strftime('%Y-%m-%d %H:%M:%S'), partner_id, url, pos, title, content, keyword, url.count('/')]
                df.loc[(len(df))] = row
            except Exception:
                print('error occured')
        except Exception:
            continue
    # part 2
    art_add = soup.find_all("article", {'class':'local-item'})
    for a in range(len(art_add)):
        h3 = art_add[a].find_all('h3', class_='local-item__name')
        title = re.sub(' +', ' ', re.sub("<b>|</b>|\'|\,|\[|\]", '',str(h3[0].find('a').contents)))
        content = re.sub(' +', ' ', re.sub("<b>|</b>|\'|\,|\[|\]", '',str(art_add[a].find('div', class_='local-item__descr').contents)[4:].lstrip()))
        url = h3[0].find_all('a', href=True)[0]['href']
        try:
            row = [datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 0, url, a + 1, title, content, keyword, url.count('/') - 2]
            df.loc[len(df)] = row
        except Exception:
            print('error occured pt2')
            return df
    return df     

#### Пример парсинга по ключевому слову 

In [5]:
keyword = 'best auto insurance'
soup = make_request(driver, keyword)

df = parse_page(soup, keyword)
df

,parse_dt,partner_id,advertiser_name,advertiser_pos,title,content,keyword,depth
0,2024-03-31 21:35:27,1,https://agenciareale.es/seguros/coche,1,"Calcula el precio del seguro de coche en"" Re...",Solicita el Precio de tu Seguro de Edificio en...,best auto insurance,4
1,2024-03-31 21:35:27,1,https://agenciareale.es/seguros/coche,1,"Calcula el precio del seguro de coche en"" Re...",Solicita el Precio de tu Seguro de Edificio en...,best auto insurance,4
2,2024-03-31 21:35:27,0,https://www.altodescuentos.com/business/detail...,1,Allstate Insurance: Kelly Qu Agency,147 W 35th St Ste 603 Nueva York NY 10001 Esta...,best auto insurance,4
3,2024-03-31 21:35:27,0,https://www.altodescuentos.com/business/detail...,2,Allegiance Financial Group,118-35 Queens Blvd Ste 400 Forest Hills NY 113...,best auto insurance,4
4,2024-03-31 21:35:27,0,https://www.altodescuentos.com/business/detail...,3,Three Star Brokerage,221 Kings Hwy Brooklyn NY 11223 Estados Unidos...,best auto insurance,4
5,2024-03-31 21:35:27,0,https://www.altodescuentos.com/business/detail...,4,Steve Fay - State Farm Insurance Agent,6608 18th Avenue Brooklyn NY 11204 Estados Uni...,best auto insurance,4
6,2024-03-31 21:35:27,0,https://www.altodescuentos.com/business/detail...,5,David J. Louie - General Insurance Agency,217 Park Row Nueva York NY 10038 Estados Unido...,best auto insurance,4
7,2024-03-31 21:35:27,0,https://www.altodescuentos.com/business/detail...,6,Virginia Chan - State Farm Insurance Agent,1015 Madison Ave Ste 503 Nueva York NY 10075 E...,best auto insurance,4
8,2024-03-31 21:35:27,0,https://www.altodescuentos.com/business/detail...,7,Michael Fay - State Farm Insurance Agent,16407 Northern Blvd Flushing NY 11358 Estados ...,best auto insurance,4
9,2024-03-31 21:35:27,0,https://www.altodescuentos.com/business/detail...,8,Full Circle Brokerage,92-11 172nd St 1FL Fl 1 Jamaica NY 11433 Estad...,best auto insurance,4
